# Locationing in Wireless Sensor Networks

Locating small devices has become feasible since Bluetooth Low Energy (BLE) has become ubiquitous. Three elements are needed:

1. The **tag** is a (small) tracking device that uses BLE to broadcast its presence to nearby devices, such as smartphones or tablets.

2. The BLE RSSI (Received **Signal Strength** Indicator), is picked up by **scanners**, devices such as smartphones that detect the BLE signals broadcast and use the RSSI to estimate the distance to the tag.

3. A **trilateration** process estimates the tag's location, based on the multiple distances calculated from the RSSI values.

Trilateration can be complemented by **fingerprinting**, in which all nearby RSSI values and the coordinates of locations are stored in a database as "fingerprints", which later can be looked up.
But: This requires the tag to constantly scan, rather than send advertisements. And advertisements are in general more energy efficient, compared to scanning for nearby packages.    

We set up a simulation example with one tag (that will move around later), and three scanning devices. These receive the advertisements from the tag at different RSSI strengths. These RSSI readings are used for three different distance measurements from the scanners to the tag.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set up the figure and axes:
def initPlot():
  plt.close('all')
  fig, ax = plt.subplots(figsize=(10, 8))
  ax.set_xlim(0, 20); ax.set_ylim(0, 20); ax.grid(True)
  # Initialize tag at (15,10), 3 scanners at (5,5),(15,15),(10,18):
  tag, = ax.plot([15], [10], 'ro', ms=10, label='Tag')
  scanners, = ax.plot([5, 15, 10], [5, 15, 18], 'bs', ms=8, label='Scanners')
  estpos, = ax.plot([15], [10], 'gx', ms=17, label='Estimated Position')
  ax.legend()
  return fig,ax,tag,scanners,estpos

initPlot()
plt.show()

The RSSI readings over the entire space in 2D can be approximated using the formula:

`Z = -50 - 20 * np.log10( np.sqrt((X - tag_x)**2 + (Y - tag_y)**2) )`

This is a basic model for estimating RSSI based on the distance between the tag and a point in space. This model is derived from the **log-distance path loss model**, which is commonly used in wireless communication to estimate signal strength as a function of distance. This is typically used as a generic model that works well in open air environments. Inside or nearby buildings, this model might not work so well.

The unit here is dBm (decibel-milliwatt), a unit of power in decibels (dB) referenced to 1 milliwatt (mW). It expresses power levels in a *logarithmic* scale, which makes it easier to compare very large and very small signal strengths in wireless communications.

In [ ]:
from matplotlib.colors import Normalize

fig,ax,tag,scanners,estpos = initPlot()

# RSSI formula:
def rssi(distance): return -50 - 20 * np.log10(distance)

def updateHeatmap(ax,X,Y,Z):
  return ax.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.5,
                     norm=Normalize(vmin=-100, vmax=-30))

# show RSSI heatmap:
X, Y = np.meshgrid( np.linspace(0, 20, 50), np.linspace(0, 20, 50) )
Z = rssi( np.sqrt((X - 15)**2 + (Y - 10)**2) )
heatmap = updateHeatmap(ax,X,Y,Z)
cbar = fig.colorbar(heatmap, ax=ax, label='RSSI (dBm)')
ax.legend()
plt.show()

Note that in this models, radio signals drop faster near the source and slower when further away. At large distances, the RSSI value approaches a "noise floor" (e.g., -100 dBm or lower), where the signal becomes indistinguishable from background noise.

`distance = np.sqrt((X - tag_x)**2 + (Y - tag_y)**2)`
calculates the Euclidean distance between the tag (at (tag_x, tag_y)) and every point (X, Y) in the 2D grid. It represents the physical distance between the tag and each point in space.

`np.log10(distance)`
takes the logarithm (base 10) of the distance. This is a key part of the log-distance path loss model, which assumes that signal strength decreases logarithmically with distance.

`-20 * np.log10(distance)`
models the path loss (how much the signal weakens as it travels through space). The factor of 20 is derived from the path loss exponent (typically 2 for free space, but can vary depending on the environment). The negative sign indicates that signal strength decreases with distance.

`-50`
is the reference RSSI value at 1 meter (also called the "transmit power" or "reference power"). It represents the signal strength (in dBm) when the receiver is 1 meter away from the transmitter (the tag). For example, if the tag broadcasts at -50 dBm at 1 meter, the RSSI will decrease as the distance increases.

In [ ]:
# Generate distance values (from 0.1m to 100m)
distances = np.linspace(0.1, 100, 500)
rssi_values = rssi(distances)

# Plot the relationship between distance and RSSI
plt.figure(figsize=(10, 5))
plt.plot(distances, rssi_values, 'b-', lw=2,
         label='RSSI = -50 - 20*log₁₀(distance)')
plt.axhline(y=-50, color='r', ls='--', label='Reference RSSI at 1m (-50 dBm)')
plt.axvline(x=1, color='g', ls='--', label='1m distance')

plt.title('RSSI vs. Distance (Log-Distance Path Loss Model)', fontsize=14)
plt.xlabel('Distance (meters)', fontsize=12)
plt.ylabel('RSSI (dBm)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=10)

# Highlight specific points and plot:
for d in [1, 2, 5, 10, 20, 50]:
    plt.scatter(d, rssi(d), color='red', zorder=5)
    plt.text(d, rssi(d) + 2, f'{d}m: {rssi(d):.1f} dBm', ha='left', fontsize=9)
plt.show()

We now show a simulation of a tag, picked up by three devices nearby that together deliver the information to locate the tag.

The heatmap visualizes the RSSI values across the 2D grid. Areas closer to the tag have higher RSSI values (e.g., -40 dBm), and areas further from the tag have lower RSSI values (e.g., -90 dBm).




In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig,ax,tag,scanners,estpos = initPlot()

def update(frame):  # update function for animation
    global heatmap, cbar
    # the tag moves in circular path:
    tag_x, tag_y = 10 + 5 * np.cos(frame * 0.1), 10 + 5 * np.sin(frame * 0.1)
    tag.set_data([tag_x], [tag_y])
    # position scanners:
    scanners.set_data([5, 15, 10], [5, 15, 18])
    # Simulate estimated position (trilateration with noise)
    estpos_x, estpos_y = tag_x + np.random.normal(0,0.5), tag_y + np.random.normal(0,0.5)
    estpos.set_data([estpos_x], [estpos_y])
    # Generate RSSI heatmap: RSSI = -50 - 20 * log10(distance)
    Z = rssi(np.sqrt((X - tag_x)**2 + (Y - tag_y)**2))
    heatmap = updateHeatmap(ax,X,Y,Z)
    # Remove old colorbar and create a new one
    cbar.remove()
    cbar = fig.colorbar(heatmap, ax=ax, label='RSSI (dBm)')
    return tag, scanners, estpos

animation = FuncAnimation(fig, update, frames=np.arange(0, 64), interval=99)
plt.close(fig)  # to avoid duplicate display
HTML(animation.to_jshtml())


Finally, the trilateration can be used to simulate the entire process:

In [ ]:
fig,ax,tag,scanners,estpos = initPlot()

def rssi2dist(rssi, A=-50, n=2):
    """RSSI to distance conversion, using a path loss model"""
    return 10 ** ((rssi - A) / (-10 * n))

def trilaterate(scanner_positions, distances):
    """trilaterate tag position in 2D"""
    # Scanner positions: [(x1,y1), (x2,y2), (x3,y3)], distances: [d1,d2,d3]
    x1, y1 = scanner_positions[0]
    x2, y2 = scanner_positions[1]
    x3, y3 = scanner_positions[2]
    d1, d2, d3 = distances
    # Solve for (x, y) using the first two scanners
    A = 2 * (x2 - x1)
    B = 2 * (y2 - y1)
    C = d1**2 - d2**2 - x1**2 + x2**2 - y1**2 + y2**2
    D = 2 * (x3 - x1)
    E = 2 * (y3 - y1)
    F = d1**2 - d3**2 - x1**2 + x3**2 - y1**2 + y3**2
    x = (C * E - F * B) / (E * A - B * D)
    y = (C * D - A * F) / (B * D - A * E)
    return x, y

def update(frame):
    global heatmap, cbar
    # Tag moves in a circular path
    tag_x = 10 + 5 * np.cos(frame * 0.1)
    tag_y = 10 + 5 * np.sin(frame * 0.1)
    tag.set_data([tag_x], [tag_y])
    # Scanner positions
    scanner_positions = [(5, 5), (15, 5), (10, 18)]
    scanners.set_data(*zip(*scanner_positions))
    # Simulate RSSI for each scanner
    distances = [np.sqrt((tag_x - sx)**2 + (tag_y - sy)**2) for sx, sy in scanner_positions]
    rssi_values = [-50 - 20 * np.log10(d) for d in distances]
    # Convert RSSI to estimated distances (with noise)
    estimated_distances = [rssi2dist(rssi) + np.random.normal(0, 0.3) for rssi in rssi_values]
    # Trilaterate
    estpos_x, estpos_y = trilaterate(scanner_positions, estimated_distances)
    estpos.set_data([estpos_x], [estpos_y])
    # Generate RSSI heatmap
    Z = np.array([[-50 - 20 * np.log10(np.sqrt((X - tag_x)**2 + (Y - tag_y)**2))]]).flatten()
    Z = Z.reshape(X.shape)
    heatmap = updateHeatmap(ax,X,Y,Z)
    # Update colorbar
    cbar.remove()
    cbar = fig.colorbar(heatmap, ax=ax, label='RSSI (dBm)')
    return tag, scanners, estpos

animation = FuncAnimation(fig, update, frames=np.arange(0, 65), interval=99)
plt.close(fig)  # to avoid duplicate display
HTML(animation.to_jshtml())